In [7]:
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely import wkt
from shapely.ops import nearest_points
import dask.dataframe as dd
import geopandas as gpd
import h3
import pandas as pd
from shapely import wkt
import pytz

In [8]:
%%time
schools = gpd.read_file("Schools_ClujNapoca.shp")
hospitals = gpd.read_file("Hospitals_ClujNapoca.shp")

CPU times: total: 93.8 ms
Wall time: 71.9 ms


In [9]:
%%time
output = "streetcsvs"

roads = gpd.read_file("RoadNetwork_ClujNapoca.shp")
schools = gpd.read_file("Schools_ClujNapoca.shp")
hospitals = gpd.read_file("Hospitals_ClujNapoca.shp")
gdf =  gpd.read_file('geo.csv')
#roads = roads[['NAME', 'COUNTY', 'Length', 'geometry']]
roads = roads.rename(columns = {'NAME': 'streetname'})

CPU times: total: 1min
Wall time: 1min 2s


In [11]:
%%time

# add desired path for data storage here

# read data into Dask
PATH = "RO_9clujnapoca_alerts.000000000000.csv.gz"
ddf = dd.read_csv(
        PATH,
        compression="gzip", 
        blocksize=None, 
        dtype={
            'confidence': 'int8',
            'type': 'category',
            'subtype': 'category',
            'roadType': 'float32', # to accept NA values
            'reliability': 'int8',
            'magvar': 'int16',
            'street': 'object'
        },
)
df = ddf.compute()

CPU times: total: 3.05 s
Wall time: 3.1 s


In [ ]:
schools

In [ ]:
merge = gdf.sjoin_nearest(roads, gdf, 'right')

In [12]:
 #where on your system do you want clean.csv?
output_path = "schools.csv"

df = pd.DataFrame(schools)
gdf = gpd.GeoDataFrame(df, geometry='geometry')
df['lon'] = gdf.geometry.x
df['lat'] = gdf.geometry.y

for i in [6,7,8,9,10]:
    df["h" + str(i)] = df.apply(lambda x: h3.geo_to_h3(x["lat"], x["lon"], i), axis=1)
df.to_csv(output_path, index=False)


In [ ]:
schools